<a href="https://colab.research.google.com/github/ymsworks0712/myacademicprojects/blob/main/Human_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python tqdm

In [ ]:
import os
import cv2
import torch
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader


In [ ]:
# Settings
# Update DATA_DIR to the correct path
DATA_DIR = "/content/drive/MyDrive/UCF-101"
IMG_SIZE = 64
FRAMES_PER_VIDEO = 10
CLASSES = ["ApplyEyeMakeup", "Archery", "BabyCrawling", "Basketball","PullUps"]  # Subset of classes
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 1. Load video frames
def extract_frames(video_path, max_frames=FRAMES_PER_VIDEO):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while True:
        ret, frame = cap.read()
        if not ret or count >= max_frames:
            break
        frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(frame)
        count += 1
    cap.release()
    # Pad with zeros if fewer frames
    while len(frames) < max_frames:
        frames.append(np.zeros((IMG_SIZE, IMG_SIZE)))
    return np.stack(frames)


In [ ]:
# 2. Custom Dataset
class HARVideoDataset(Dataset):
    def __init__(self, root_dir, classes):
        self.data = []
        self.labels = []
        self.class_to_idx = {cls: i for i, cls in enumerate(classes)}

        for cls in classes:
            cls_dir = os.path.join(root_dir, cls)
            for file in os.listdir(cls_dir)[:50]:
                video_path = os.path.join(cls_dir, file)
                frames = extract_frames(video_path)
                self.data.append(frames)
                self.labels.append(self.class_to_idx[cls])

        self.data = np.array(self.data, dtype=np.float32) / 255.0
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        video = self.data[idx].reshape(-1)  # Flatten frames
        label = self.labels[idx]
        return torch.tensor(video), torch.tensor(label)

In [ ]:
class AdvancedHARNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(AdvancedHARNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(hidden_size // 2, num_classes)
        )

   def forward(self, x):
        return self.net( x)


In [ ]:
# 4. Load data
dataset = HARVideoDataset(DATA_DIR, CLASSES)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [ ]:
# 5. Initialize model
input_size = IMG_SIZE * IMG_SIZE * FRAMES_PER_VIDEO  # Flattened video
hidden_size = 512
num_classes = len(CLASSES)

model = AdvancedHARNet(input_size=40960, hidden_size=512, num_classes=len(CLASSES)).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [ ]:
# 6. Training
print("Training...")
for epoch in range(70):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Training...
Epoch 1, Loss: 0.1796
Epoch 2, Loss: 0.0286
Epoch 3, Loss: 0.0282
Epoch 4, Loss: 0.0853
Epoch 5, Loss: 0.0712
Epoch 6, Loss: 0.0563
Epoch 7, Loss: 0.0178
Epoch 8, Loss: 0.0750
Epoch 9, Loss: 0.2162
Epoch 10, Loss: 0.0219
Epoch 11, Loss: 0.1016
Epoch 12, Loss: 0.0168
Epoch 13, Loss: 0.0651
Epoch 14, Loss: 0.1197
Epoch 15, Loss: 0.0224
Epoch 16, Loss: 0.0079
Epoch 17, Loss: 0.1631
Epoch 18, Loss: 0.0081
Epoch 19, Loss: 0.0197
Epoch 20, Loss: 0.0198
Epoch 21, Loss: 0.4269
Epoch 22, Loss: 0.0152
Epoch 23, Loss: 0.1142
Epoch 24, Loss: 0.2201
Epoch 25, Loss: 0.0653
Epoch 26, Loss: 0.0426
Epoch 27, Loss: 0.0836
Epoch 28, Loss: 0.0264
Epoch 29, Loss: 0.0180
Epoch 30, Loss: 0.0345
Epoch 31, Loss: 0.0947
Epoch 32, Loss: 0.0133
Epoch 33, Loss: 0.0669
Epoch 34, Loss: 0.0332
Epoch 35, Loss: 0.1770
Epoch 36, Loss: 0.0290
Epoch 37, Loss: 0.0582
Epoch 38, Loss: 0.0365
Epoch 39, Loss: 0.0462
Epoch 40, Loss: 0.2841
Epoch 41, Loss: 0.0224
Epoch 42, Loss: 0.1640
Epoch 43, Loss: 0.0519
Epoch 44

In [ ]:
# 7. Testing
print("\nEvaluating...")
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Evaluating...
Test Accuracy: 72.00%


In [ ]:
def predict_video_class(model, video_path, classes):
    model.eval()  # Taking model to evaluation mode

    # Finding Frame from Videos
    frames = extract_frames(video_path)
    frames = frames / 255.0  # normalize
    frames = torch.tensor(frames, dtype=torch.float32)

    # Making frames 1D vector
    input_tensor = frames.view(1, -1)  # shape: [1, 40960]
    input_tensor = input_tensor.to(DEVICE)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted_class = torch.max(outputs, 1)

    # Showing Class Name
    print("🎬 Predicted Activity:", classes[predicted_class.item()])

In [ ]:
# Example : Giving a Videopath from Dataset
sample_video_path = "/content/drive/MyDrive/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g03_c02.avi"

predict_video_class(model, sample_video_path, CLASSES)

🎬 Predicted Activity: ApplyEyeMakeup
